### Introduction

When we talk about **DevOps**, we could more likely to think about **CI/CD**, so this tutorial is try to use GCP to make the **CI/CD** work in cloud. During this notebook, we will also meet **cloud source repository**(a tool just like **Github**, but not support all features.), **Cloud Build**(to use it to do auto build for whole source code in repository) and **Cloud function** etc.

Let's get start first.

In [6]:
# before we do anything we should auth this notebook
! gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=VpAjFc0DiKqpvx9HmD3NII87tH547BVOTFLU7AzCyJs&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/1AHnt9DNoSdaMx0ysrV19iSPMr3xZBU_ZSUISbOfUDtdckDQaftt_OE

You are now logged in as [gqianglu1990@gmail.com].
Your current project is [cloudtutorial-279003].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [7]:
# first set project
! gcloud config set project cloudtutorial-279003

Updated property [core/project].


In [8]:
# first we will use cloud source repository to store our source code.

# so first let's create a repository in remote server first
! gcloud source repos create lugq-repos

Created [lugq-repos].


In [9]:
# then clone the remote repos into our local server
! gcloud source repos clone lugq-repos

Cloning into '/content/lugq-repos'...
Project [cloudtutorial-279003] repository [lugq-repos] was cloned to [/content/lugq-repos].


In [11]:
# let's create our first python app.
import os

# change to the repository folder
os.chdir("lugq-repos")

In [13]:
# we could create a flask app
%%writefile main.py
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
  return "Hello, this is lugq."

if __name__ == "__main__":
  app.run(debug=False)

Overwriting main.py


In [29]:
# as we will deploy our code into containers, so let's first create a app.yaml file
# to define the runtime of our code
%%writefile app.yaml
runtime: python27
api_version: 1
threadsafe: yes

handlers:
- url: .*
  script: main.app

libraries:
- name: flask
  version: '0.12'

Overwriting app.yaml


In [14]:
import flask
flask.__version__

'1.1.2'

In [18]:
# where we are.
os.path.abspath(os.curdir)

'/content/lugq-repos'

In [20]:
# before we do anything about git, we have to tell the current running shell the git info.
! git config --global user.email "gqianglu1990@gmail.com"
! git config --global user.name "gqianglu1990"

In [30]:
# then that's our git add/ commit /push
! git add .

! git commit -m "libraries entries are only supported by the python27 runtime"

! git push origin master

[master 94363b6] libraries entries are only supported by the python27 runtime
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 348 bytes | 348.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1)
To https://source.developers.google.com/p/cloudtutorial-279003/r/lugq-repos
   7534a50..94363b6  master -> master


In [32]:
# let's deploy our code into application app
# you have to ensure the app engine is enabled,  but if we finished the whole thing, we should disable it.
! gcloud app deploy app.yaml

Services to deploy:

descriptor:      [/content/lugq-repos/app.yaml]
source:          [/content/lugq-repos]
target project:  [cloudtutorial-279003]
target service:  [default]
target version:  [20200622t092140]
target url:      [https://cloudtutorial-279003.uc.r.appspot.com]


Do you want to continue (Y/n)?  y

Beginning deployment of service [default]...
╔════════════════════════════════════════════════════════════╗
╠═ Uploading 3 files to Google Cloud Storage                ═╣
╚════════════════════════════════════════════════════════════╝
File upload done.
Deployed service [default] to [https://cloudtutorial-279003.uc.r.appspot.com]

You can stream logs from the command line by running:
  $ gcloud app logs tail -s default

To view your application in the web browser run:
  $ gcloud app browse


In [33]:
# let's check our appliation
! gcloud app browse

# under link we get:  Hello, this is lugq.
# so we do deploy our code into the application

Did not detect your browser. Go to this link to view your app:
https://cloudtutorial-279003.uc.r.appspot.com


### CI/CD with Cloud Build


In [34]:
# first we have to create a file called: cloudbuild.yaml to describe the step we want to use.
%%writefile cloudbuild.yaml

steps:
- name: 'gcr.io/cloud-builders/gcloud'
  args: ["app", "deploy"]

timeout: "60s"

Writing cloudbuild.yaml


In [35]:
# let's add our cloudbuild.yaml into repository
! git add .

! git commit -m "add cloudbuild.yaml file"

! git push origin master

[master 16b1a29] add cloudbuild.yaml file
 1 file changed, 6 insertions(+)
 create mode 100644 cloudbuild.yaml
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 398 bytes | 398.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://source.developers.google.com/p/cloudtutorial-279003/r/lugq-repos
   94363b6..16b1a29  master -> master


#### Next step in cloud build

As next step should be finished in **Cloud Build** console, you could find it [here](https://cloud.google.com/source-repositories/docs/quickstart-triggering-builds-with-source-repositories#create_a_build_trigger).

I have created a trigger event called: `app-trigger`, then let's change our code a bit, then we will don't need to step by step again with above steps. 

Let's test.

In [36]:
# change main file
%%writefile main.py
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
  return "Hello, this is lugq. Here I just add the CI/CD with cloud build."

if __name__ == "__main__":
  app.run(debug=False)

Overwriting main.py


In [37]:
# if we need to trigger our cloud build, we have to upload our files into git, so
# that cloud build could moniter the git source file, if there are changes happen, 
# will trigger our `cloudbuild.yaml` file.

! git add .

! git commit -m "change main file for CI/CD"

! git push origin master

[master 12d3c0d] change main file for CI/CD
 1 file changed, 1 insertion(+), 1 deletion(-)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 334 bytes | 334.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2)
To https://source.developers.google.com/p/cloudtutorial-279003/r/lugq-repos
   16b1a29..12d3c0d  master -> master


### Tips

You could get the latest status of **cloud build** in **history** tab, after you have seen it finished, let's check again our app.

In [38]:
! gcloud app browse

# this is what I get from bellow link:  `Hello, this is lugq. Here I just add the CI/CD with cloud build.`

# so we do get the lasted change without any manually deploy step.

Did not detect your browser. Go to this link to view your app:
https://cloudtutorial-279003.uc.r.appspot.com


### Cloud function with repository

In [45]:
try:
  os.mkdir("cloud_func")
except:
  print("Create folder with error.")

In [46]:
#  make a hello_world function for repository
%%writefile cloud_func/main.py

def hello_world(request):
  # convert request into json
  request_json = request.get_json()

  if request.args and 'message' in request.args:
    return request.args.get("message")
  elif request_json and 'message' in request_json:
    return request_json['message']
  else:
    return "hello lu"

Writing cloud_func/main.py


In [47]:
# let's upload the file into git
! git add .

! git commit -m "change with cloud function in new folder"

! git push origin master

[master 86da3d6] change with cloud function in new folder
 1 file changed, 0 insertions(+), 0 deletions(-)
 rename "cloud_func\\main.py" => cloud_func/main.py (100%)
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 394 bytes | 394.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0)
To https://source.developers.google.com/p/cloudtutorial-279003/r/lugq-repos
   599d51e..86da3d6  master -> master


#### Deploy cloud function with repository

We could follow the step to create a cloud function from repository like [this](https://cloud.google.com/source-repositories/docs/quickstart-integrating-with-cloud-functions#create_and_deploy_the_function).

You could see the status of cloud function in the project console cloud functions.


**Tips**

If you face any issue about 403 forbidden, then you could get solution [here](https://stackoverflow.com/questions/47511677/firebase-cloud-function-your-client-does-not-have-permission-to-get-url-200-fr), that's because of cloud function authentication error.

In [53]:
# after the function is deployed, then we could try to trigger our function with requests
import requests

trigger_url = "https://us-central1-cloudtutorial-279003.cloudfunctions.net/cloud-repos"

def post_data(url, data=None):
  res = requests.post(url, json=data)
  print("Data has been posted, get response result:  {}".format(res.text))

post_data(trigger_url, {'message': "lu"})

Data has been posted, get response result:  lu


In [54]:
# so we have get the result from cloud function, let's remove our function
! gcloud functions delete cloud-repos

Resource [projects/cloudtutorial-279003/locations/us-central1/function
s/cloud-repos] will be deleted.

Do you want to continue (Y/n)?  y

Deleted [projects/cloudtutorial-279003/locations/us-central1/functions/cloud-repos].
